In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
os.chdir("C:\MSBAPM\Python\Voya\Voya_Final_Data")

In [2]:
df=pd.read_csv("Final_Subject.csv")
df.head()

,Subject,Sent_Rate,S_Rate,Rate,Successful
0,Utilizing 529 Plans during Tax Season,58735,7736.0,0.131710,0
1,College Savings can be a Slam Dunk with Tomorr...,58610,11054.0,0.188603,0
2,Join Us! Global Perspectives Midyear Outlook W...,53597,9201.0,0.171670,0
3,Join Us! Global Perspectives Quarterly Market ...,50943,9005.0,0.176766,0
4,Reminder: Global Perspectives 2018 Forecast We...,48476,3451.0,0.071190,0


In [5]:
df.loc[df['Rate']>0.21,'Successful']=1

In [6]:
df.drop(['Sent_Rate','S_Rate','Rate'],axis=1,inplace=True)

In [7]:
df.head()

,Subject,Successful
0,Utilizing 529 Plans during Tax Season,0
1,College Savings can be a Slam Dunk with Tomorr...,0
2,Join Us! Global Perspectives Midyear Outlook W...,0
3,Join Us! Global Perspectives Quarterly Market ...,0
4,Reminder: Global Perspectives 2018 Forecast We...,0


In [6]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 876):
    Subject = re.sub('[^a-zA-Z]', ' ', df['Subject'][i])
    Subject = Subject.lower()
    Subject = Subject.split()
    ps = PorterStemmer()
    Subject = [ps.stem(word) for word in Subject if not word in set(stopwords.words('english'))]
    Subject = ' '.join(Subject)
    corpus.append(Subject)

In [7]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1000)
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:, 1].values

In [8]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [9]:
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [10]:
# Predicting the Test set results
y_pred1 = classifier.predict(X_test)

In [11]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred1)

In [12]:
cm

array([[ 97,  16],
       [100,  50]], dtype=int64)

In [13]:
n_feat=1000

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

def get_model(n_feat):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=n_feat))
    model.add(Dense(128, activation="relu"))
    #model.add(Dropout(.2))
    model.add(Dense(256, activation="relu"))
    #model.add(Dropout(.5))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

C:\Users\chara\Downloads\ANACONDA\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [15]:
NN = get_model(n_feat)
NN.fit(X_train,y_train, epochs=10)

Epoch 1/10
613/613 [==============================] - 1s 2ms/step - loss: 0.6872 - acc: 0.5954
Epoch 2/10
613/613 [==============================] - 0s 106us/step - loss: 0.6313 - acc: 0.7684
Epoch 3/10
613/613 [==============================] - 0s 108us/step - loss: 0.4591 - acc: 0.8271
Epoch 4/10
613/613 [==============================] - 0s 111us/step - loss: 0.3033 - acc: 0.8760
Epoch 5/10
613/613 [==============================] - 0s 112us/step - loss: 0.2090 - acc: 0.9135
Epoch 6/10
613/613 [==============================] - 0s 109us/step - loss: 0.1655 - acc: 0.9315
Epoch 7/10
613/613 [==============================] - 0s 111us/step - loss: 0.1278 - acc: 0.9511
Epoch 8/10
613/613 [==============================] - 0s 106us/step - loss: 0.1230 - acc: 0.9445
Epoch 9/10
613/613 [==============================] - 0s 106us/step - loss: 0.1168 - acc: 0.9478
Epoch 10/10
613/613 [==============================] - 0s 121us/step - loss: 0.0947 - acc: 0.9576


In [16]:
from sklearn.metrics import classification_report

def predict(model,X_test):
    y_pred = model.predict(X_test)
    return [1 if pred >= .5 else 0 for pred in y_pred]
def print_report(model, x, y_true):
    y_pred = predict(model, X_test)
    report = classification_report(y_test, y_pred)
    print(report)

In [17]:
print_report(classifier,X_test,y_test)

              precision    recall  f1-score   support

           0       0.49      0.86      0.63       113
           1       0.76      0.33      0.46       150

   micro avg       0.56      0.56      0.56       263
   macro avg       0.62      0.60      0.54       263
weighted avg       0.64      0.56      0.53       263



In [18]:
print_report(NN,X_test,y_test)

              precision    recall  f1-score   support

           0       0.68      0.70      0.69       113
           1       0.77      0.75      0.76       150

   micro avg       0.73      0.73      0.73       263
   macro avg       0.72      0.72      0.72       263
weighted avg       0.73      0.73      0.73       263



In [19]:
y_pred1=predict(NN,X_test)

In [20]:
y_pred1=np.array(y_pred1)

In [21]:
y_pred=predict(classifier,X_test)

In [22]:
confusion_matrix(y_test,y_pred)

array([[ 97,  16],
       [100,  50]], dtype=int64)

In [23]:
confusion_matrix(y_test,y_pred1)

array([[ 79,  34],
       [ 38, 112]], dtype=int64)

In [24]:
y_test.shape

(263,)

In [25]:
187/263

0.7110266159695817

In [26]:
76/263

0.2889733840304182

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

In [8]:
import re
import string
import nltk

def cleanup(sentence):
    sentence = sentence.lower()
    sentence = re.sub("[\[].*?[\]]", "", sentence)
    return sentence

df['Words'] = df['Subject'].apply(cleanup)

In [9]:
count_vect = CountVectorizer(min_df = 1, max_features = 500)
X_train_counts = count_vect.fit_transform(df['Words'])

X_train_counts = X_train_counts.toarray()
print(X_train_counts.shape)

# Take a look at the words in the vocabulary
vocab = count_vect.get_feature_names()
print(vocab)

(876, 500)
['10', '100', '11', '12', '13', '16', '17', '19', '20', '2016', '2017', '2018', '21', '23', '24', '26', '27th', '30', '31', '529', 'about', 'access', 'across', 'active', 'activity', 'adds', 'advisors', 'advisory', 'after', 'again', 'agg', 'all', 'alternate', 'always', 'an', 'and', 'another', 'anticipating', 'approach', 'april', 'are', 'as', 'asset', 'assets', 'at', 'august', 'available', 'avoid', 'back', 'balance', 'bank', 'be', 'bear', 'best', 'better', 'blog', 'bond', 'bonds', 'book', 'build', 'bull', 'bump', 'business', 'but', 'buy', 'by', 'call', 'can', 'canâ', 'cap', 'capital', 'capture', 'care', 'case', 'chance', 'change', 'changing', 'china', 'choppy', 'chura', 'cio', 'client', 'clients', 'climb', 'college', 'come', 'coming', 'commentary', 'common', 'conference', 'consistency', 'consistent', 'continue', 'continues', 'core', 'credit', 'cut', 'cuts', 'cycle', 'data', 'date', 'day', 'days', 'december', 'design', 'despite', 'diligence', 'dipped', 'diversification', 'diver

In [10]:
# Sum up the counts of each vocabulary word
dist = np.sum(X_train_counts, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
a=[]
for tag, count in zip(vocab, dist):
    a=(count, tag)

In [11]:
G=pd.DataFrame(vocab)

In [12]:
G['Count']=dist

In [13]:
G=G.rename({0:'word'},axis=1)

In [14]:
G.head()

,word,Count
0,10,3
1,100,4
2,11,5
3,12,6
4,13,3


In [15]:
G.tail()

,word,Count
495,years,3
496,yield,3
497,yields,3
498,you,49
499,your,46


In [16]:
G.to_csv('Subject_WordCount.csv',index=False)